In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

# 1. DenseNet121을 사용한 이미지 특징 추출기
def build_feature_extractor(input_shape=(256, 256, 3)):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# 2. 이미지에서 특징 벡터 추출
def extract_features(model, images):
    return model.predict(images)

# 3. XGBoost 모델을 GridSearchCV로 학습
def train_xgboost(features, temp_humid, labels):
    # 이미지 특징 벡터와 온도/습도 데이터를 결합
    X = np.concatenate([features, temp_humid], axis=1)

    # 하이퍼파라미터 그리드 설정
    param_grid = {
        'colsample_bytree': [0.3, 0.7],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'alpha': [10, 20],
        'n_estimators': [100, 200, 300]
    }

    # XGBoost 모델 초기화
    xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

    # GridSearchCV로 최적 하이퍼파라미터 탐색
    grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid,
                               scoring='neg_mean_squared_error', cv=5, verbose=1)

    # 모델 학습
    grid_search.fit(X, labels)

    # 최적 하이퍼파라미터 출력
    print("Best parameters found: ", grid_search.best_params_)

    # 최적 하이퍼파라미터로 학습된 모델 반환
    return grid_search.best_estimator_

# 데이터 경로 설정
image_base_path = '/content/day'  # 이미지가 저장된 상위 폴더의 경로
folders = ['1', '2', '3', '4', '5', '6', '7']

# 온도와 습도 데이터
temp_humid_data = np.array([
    [23.59, 57.18],
    [24.21, 52.13],
    [24.87, 55.69],
    [25.28, 56.29],
    [27.29, 54.21],
    [25.28, 56.29],
    [27.29, 54.21],
    [24.89, 59.31],
    [26.45, 53.35]
])

# 각 폴더 내 이미지를 불러오고 라벨을 연결
images = []
labels = []
temp_humid_features = []

for i, folder in enumerate(folders):
    folder_path = os.path.join(image_base_path, folder)
    for image_name in os.listdir(folder_path):
        # 이미지 로드 및 전처리
        image_path = os.path.join(folder_path, image_name)
        image = load_img(image_path, target_size=(256, 256))
        image_array = img_to_array(image)
        images.append(image_array)

        # 라벨 추가 (해당 폴더가 일수 라벨임)
        labels.append(int(folder))

        # 해당 이미지에 대응하는 온도 및 습도 추가 (임의로 매핑된 데이터 사용)
        temp_humid_features.append(temp_humid_data[i % len(temp_humid_data)])

# numpy 배열로 변환
images = np.array(images)
labels = np.array(labels)
temp_humid_features = np.array(temp_humid_features)

# 확인
print(f"Total images loaded: {len(images)}")
print(f"Labels: {labels.shape}")
print(f"Temperature and Humidity Features: {temp_humid_features.shape}")

# DenseNet121 모델 불러오기
feature_extractor = build_feature_extractor()

# 이미지에서 특징 벡터 추출
features = extract_features(feature_extractor, images)

# XGBoost 모델 학습
xgboost_model = train_xgboost(features, temp_humid_features, labels)

# 예측에 사용할 이미지 경로와 온도/습도 데이터
image_path = '/content/dataset2_val_IMG_20191215_111318_4.jpg'  # 예측할 이미지의 경로
temp_humid = np.array([[25.67, 54.68]])  # 예측할 때 사용할 온도 및 습도 데이터

# 이미지 로드 및 전처리
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = np.expand_dims(image_array, axis=0)  # 배치 차원 추가

# DenseNet121을 사용하여 이미지 특징 추출
new_features = extract_features(feature_extractor, image_array)

# 이미지 특징 벡터와 온도/습도 데이터를 결합
input_data = np.concatenate([new_features, temp_humid], axis=1)

# XGBoost 모델을 사용하여 예측
predicted_days = xgboost_model.predict(input_data)

print("Predicted days until ripe:", predicted_days)


Total images loaded: 641
Labels: (641,)
Temperature and Humidity Features: (641, 2)
21/21 ━━━━━━━━━━━━━━━━━━━━ 148s 7s/step
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found:  {'alpha': 10, 'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
Predicted days until ripe: [3.9572146]


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 예측에 사용할 이미지 경로와 온도/습도 데이터
image_path = '/content/dataset2_train_IMG_20191215_110931_1.jpg'  # 예측할 이미지의 경로
temp_humid = np.array([[25.67, 54.68]])  # 예측할 때 사용할 온도 및 습도 데이터

# 이미지 로드 및 전처리
image = load_img(image_path, target_size=(256, 256))
image_array = img_to_array(image)
image_array = np.expand_dims(image_array, axis=0)  # 배치 차원 추가

# DenseNet121을 사용하여 이미지 특징 추출
new_features = extract_features(feature_extractor, image_array)

# 이미지 특징 벡터와 온도/습도 데이터를 결합
input_data = np.concatenate([new_features, temp_humid], axis=1)

# XGBoost 모델을 사용하여 예측
predicted_days = xgboost_model.predict(input_data)

print("Predicted days until ripe:", predicted_days)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
Predicted days until ripe: [3.9853168]
